In [1]:
import requests
import pandas as pd
import plotly.graph_objects as go

In [2]:
class GithubIssues:
    """
    Initial code for API.

    GithubIssues.set_repos accepts a list of the form:
    ['filetrust/mvp-icap-service',
     'filetrust/mvp-icap-cloud',
      'filetrust/mvp-icap-squid-cache-proxy',
      'filetrust/rebuild-k8s-filetypedetection',
      'filetrust/icap-performance-tests',
      'filetrust/rebuild-k8s']

    Github.plot_totals will generate a simple Card visual with a repo slicer dropdown.
    """
    def __init__(self):
        self._api_url = 'https://api.github.com/repos/'
        self._df_list = None

    def _issue_response(self,repo):
        """
        Returns a json list of issues for a specified repo.
        """
        response = requests.get(repo.join((self._api_url,'/issues')))
        if response.ok:
            return response
        else:
            raise Exception(' '.join(('Repo',repo,'not found.')))

    def _issue_json(self,repo):
        """
        Returns a json list of issues for a specified repo.
        """
        return self._issue_response(repo).json()   

    def _set_df_list(self,repo_list):
        """
        Returns a list of Pandas DataFrames of issues for all repos.
        """
        self._df_list= [pd.DataFrame(self._issue_json(repo)) for repo in repo_list]

    def set_repos(self,repo_list):
        """
        Accepts a list of repos of the form:
        ['filetrust/mvp-icap-service',
        'filetrust/mvp-icap-cloud',
        'filetrust/mvp-icap-squid-cache-proxy',
        'filetrust/rebuild-k8s-filetypedetection',
        'filetrust/icap-performance-tests',
        'filetrust/rebuild-k8s']
        """
        self.repo_list=repo_list
        self._set_df_list(repo_list)
        return self

    def plot_total(self):
        """

        """
        off_white = 'rgb(200,200,200)'
        dark_grey = 'rgb(40,40,40)'

        if self._df_list is not None:
            fig=go.Figure()
            for n,repo,df in zip(range(len(self._df_list)),self.repo_list,self._df_list):
                fig.add_trace(go.Indicator(mode = "number+delta",number={'suffix': ' Issues'},
                value = len(df),title = {"text": 'Repo: ' + repo}, delta={'reference':0},visible = n==0))

            fig.update_layout(
            updatemenus=[
                dict(
                    active=0,
                    buttons=list([
                        dict(label=repo,
                            method="update",
                            args=[{"visible": [i==j for j in range(len(self.repo_list))]}]) for i,repo in enumerate(self.repo_list)
                    ]),  x=0.75,y=2.0,
                ), 
            ],paper_bgcolor=dark_grey,font={'color':off_white})

            return fig
        else:
            raise Exception('set_repos needs to be called before visuals can be created')



In [3]:
# Initial repo_list used for development.
repo_list = ['filetrust/mvp-icap-service',
            'filetrust/mvp-icap-cloud',
            'filetrust/mvp-icap-squid-cache-proxy',
            'filetrust/rebuild-k8s-filetypedetection',
            'filetrust/icap-performance-tests',
            'filetrust/rebuild-k8s']
GithubIssues().set_repos(repo_list).plot_total()